In [3]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# autoreload chaque module spécifié par %aimport
%autoreload 1

%aimport src.kerasCallbacks
%aimport src.results

from IPython.display import display
from os import path
import time

from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.utils import to_categorical

from src import constants
from src.kerasCallbacks import BestWeightsRestorer
import src.results as resultsUtil
import src.runUtil as runUtil

In [5]:

def load_feature_set():

    file_path = path.join(constants.RAW_TAGGED_FEATURE_SET_PATH, "msd-marsyas_dev_new/msd-marsyas_dev_new.csv")
    
    ftrs = np.array(pd.read_csv(file_path, header=None).values[:,2:-1])
    lbls = np.array(pd.read_csv(file_path, header=None).values[:,-1])
    
    return ftrs, lbls

def scale_features(ftrs):
    
    standardscaler = StandardScaler()
    ftrs_scld = standardscaler.fit_transform(ftrs)
    
    return ftrs_scld

def one_hot_labels(lbls):
    
    lbls_1d = lbls.reshape(len(lbls), 1)
        
    oh_encoder = OneHotEncoder(sparse=False)
    lbls_oh = oh_encoder.fit_transform(lbls_1d)
    
    return lbls_oh

In [6]:
# Load the features and the labels
raw_ftrs, raw_lbls = load_feature_set()

lbl_encoder = LabelEncoder()
lbl_encoded_lbls = lbl_encoder.fit_transform(raw_lbls)

ftrs = scale_features(raw_ftrs)

In [7]:
def create_model(input_shape):
    
    inputs = Input(shape=input_shape)
    
    layer = Dense(90, activation='relu')(inputs)
    layer = Dense(85, activation='relu')(layer)
    
    layer = Dropout(0.10)(layer)
    
    layer = Dense(80, activation='relu')(layer)
    layer = Dense(75, activation='relu')(layer)
    layer = Dense(70, activation='relu')(layer)
    
    layer = BatchNormalization()(layer)
    
    layer = Dense(65, activation='relu')(layer)
    layer = Dense(60, activation='relu')(layer)
    
    layer = BatchNormalization()(layer)
    
    layer = Dense(55, activation='relu')(layer)
    layer = Dense(50, activation='relu')(layer)
    
    layer = Dropout(0.10)(layer)
    
    layer = Dense(45, activation='relu')(layer)
    layer = Dense(40, activation='relu')(layer)
    
    layer = Dropout(0.10)(layer)
    
    layer = Dense(30, activation='relu')(layer)
    
    outputs = Dense(25, activation='softmax')(layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=Adam(lr=0.001), 
        loss='categorical_crossentropy', 
        metrics=[
            CategoricalAccuracy(name='accuracy')
        ]
    )
    
    return model

In [8]:
experiment_number = 4

In [14]:
def train_fold(x_train, y_train, x_val, y_val, run_name, fold=None):

    input_shape = x_train[0].shape
    model = create_model(input_shape)
    
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    # Resample the train data to overcome imbalance
    x_train_resampl, y_train_resampl = SMOTE().fit_resample(x_train, y_train)
    
    if fold is not None:
        run_name += '_fold' + str(fold)
        print('Training fold ' + str(fold))
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    weight_restorer = BestWeightsRestorer(monitor='val_loss', mode='min', verbose=1)
    
    history = model.fit(
        x=x_train_resampl,
        y=y_train_resampl,
        validation_data=[x_val, y_val],
        epochs=110,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board, weight_restorer],
        verbose=1
    )
    
    prob_predictions = model.predict(x_val)
    predictions = prob_predictions.argmax(axis=1)
    
    return {
        'history': history.history,
        'predictions': predictions
    }

experiment_number += 1
run_name = 'base_mlp_marsyas_cv_balanced_' + str(experiment_number)

results = resultsUtil.cross_validate(
    x=ftrs,
    y=lbl_encoded_lbls,
    n_splits=5,
    train_func=train_fold,
    run_name=run_name
)

runUtil.save_run_results(run_name, results)

resultsdf = pd.DataFrame([results]).transpose()

display(resultsdf)

Training fold 1
Train on 237150 samples, validate on 35922 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/110
237150/237150 [==============================] - 2s 11us/sample - loss: 2.8867 - accuracy: 0.1478 - val_loss: 2.8725 - val_accuracy: 0.1659
Epoch 2/110
237150/237150 [==============================] - 1s 6us/sample - loss: 2.5356 - accuracy: 0.2318 - val_loss: 2.6070 - val_accuracy: 0.2151
Epoch 3/110
237150/237150 [==============================] - 1s 6us/sample - loss: 2.4399 - accuracy: 0.2604 - val_loss: 2.4758 - val_accuracy: 0.2428
Epoch 4/110
237150/237150 [==============================] - 1s 6us/sample - loss: 2.3800 - accuracy: 0.2789 - val_loss: 2.4532 - val_accuracy: 0.2447
Epoch 5/110
237150/237150 [==============================] - 1s 6us/sample - loss: 2.3366 - accuracy: 0.2940 - val_loss: 2.4374 - val_accuracy: 0.2522
Epoch 6/110
237150/237150 [==============================] - 1s 6us/sample - loss: 2.2991 - accuracy: 0.3065 - val_loss: 2.4256 -

Epoch 53/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9904 - accuracy: 0.3981 - val_loss: 2.3355 - val_accuracy: 0.2900
Epoch 54/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9884 - accuracy: 0.3993 - val_loss: 2.3443 - val_accuracy: 0.2867
Epoch 55/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9872 - accuracy: 0.4001 - val_loss: 2.3427 - val_accuracy: 0.2844
Epoch 56/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9837 - accuracy: 0.4015 - val_loss: 2.3343 - val_accuracy: 0.2906
Epoch 57/110
237150/237150 [==============================] - 1s 5us/sample - loss: 1.9851 - accuracy: 0.3998 - val_loss: 2.3418 - val_accuracy: 0.2890
Epoch 58/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9825 - accuracy: 0.4011 - val_loss: 2.3464 - val_accuracy: 0.2874
Epoch 59/110
237150/237150 [==============================] - 1s 5us/sample - loss: 1.97

Epoch 107/110
237150/237150 [==============================] - 1s 5us/sample - loss: 1.9218 - accuracy: 0.4188 - val_loss: 2.3634 - val_accuracy: 0.2839
Epoch 108/110
237150/237150 [==============================] - 1s 5us/sample - loss: 1.9179 - accuracy: 0.4211 - val_loss: 2.3546 - val_accuracy: 0.2884
Epoch 109/110
237150/237150 [==============================] - 1s 5us/sample - loss: 1.9187 - accuracy: 0.4197 - val_loss: 2.3528 - val_accuracy: 0.2864
Epoch 110/110
237150/237150 [==============================] - 1s 5us/sample - loss: 1.9170 - accuracy: 0.4210 - val_loss: 2.3587 - val_accuracy: 0.2884
Restoring the best weights from epoch 110
Training fold 2
Train on 237150 samples, validate on 35914 samples
Epoch 1/110
237150/237150 [==============================] - 2s 10us/sample - loss: 2.8796 - accuracy: 0.1448 - val_loss: 2.8589 - val_accuracy: 0.1728
Epoch 2/110
237150/237150 [==============================] - 1s 6us/sample - loss: 2.5266 - accuracy: 0.2360 - val_loss: 2.5786

237150/237150 [==============================] - 1s 6us/sample - loss: 1.9928 - accuracy: 0.3965 - val_loss: 2.3496 - val_accuracy: 0.2889
Epoch 51/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9908 - accuracy: 0.3972 - val_loss: 2.3485 - val_accuracy: 0.2886
Epoch 52/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9918 - accuracy: 0.3968 - val_loss: 2.3445 - val_accuracy: 0.2890
Epoch 53/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9906 - accuracy: 0.3978 - val_loss: 2.3506 - val_accuracy: 0.2877
Epoch 54/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9862 - accuracy: 0.3993 - val_loss: 2.3633 - val_accuracy: 0.2865
Epoch 55/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9831 - accuracy: 0.3998 - val_loss: 2.3387 - val_accuracy: 0.2899
Epoch 56/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9810 - accuracy

Epoch 104/110
237150/237150 [==============================] - 1s 5us/sample - loss: 1.9213 - accuracy: 0.4176 - val_loss: 2.3678 - val_accuracy: 0.2840
Epoch 105/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9234 - accuracy: 0.4177 - val_loss: 2.3544 - val_accuracy: 0.2879
Epoch 106/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9221 - accuracy: 0.4168 - val_loss: 2.3713 - val_accuracy: 0.2867
Epoch 107/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9187 - accuracy: 0.4190 - val_loss: 2.3674 - val_accuracy: 0.2883
Epoch 108/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9192 - accuracy: 0.4191 - val_loss: 2.3716 - val_accuracy: 0.2889
Epoch 109/110
237150/237150 [==============================] - 1s 6us/sample - loss: 1.9197 - accuracy: 0.4175 - val_loss: 2.3557 - val_accuracy: 0.2887
Epoch 110/110
237150/237150 [==============================] - 1s 6us/sample - los

...

237175/237175 [==============================] - 1s 6us/sample - loss: 2.0078 - accuracy: 0.3928 - val_loss: 2.3540 - val_accuracy: 0.2863
Epoch 45/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0021 - accuracy: 0.3951 - val_loss: 2.3536 - val_accuracy: 0.2847
Epoch 46/110
237175/237175 [==============================] - 1s 5us/sample - loss: 2.0012 - accuracy: 0.3954 - val_loss: 2.3496 - val_accuracy: 0.2871
Epoch 47/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0017 - accuracy: 0.3942 - val_loss: 2.3565 - val_accuracy: 0.2848
Epoch 48/110
237175/237175 [==============================] - 1s 5us/sample - loss: 1.9954 - accuracy: 0.3965 - val_loss: 2.3581 - val_accuracy: 0.2863
Epoch 49/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9944 - accuracy: 0.3967 - val_loss: 2.3506 - val_accuracy: 0.2875
Epoch 50/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9938 - accuracy

Epoch 98/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9259 - accuracy: 0.4158 - val_loss: 2.3537 - val_accuracy: 0.2861
Epoch 99/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9229 - accuracy: 0.4171 - val_loss: 2.3678 - val_accuracy: 0.2844
Epoch 100/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9235 - accuracy: 0.4173 - val_loss: 2.3672 - val_accuracy: 0.2872
Epoch 101/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9206 - accuracy: 0.4172 - val_loss: 2.3617 - val_accuracy: 0.2859
Epoch 102/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9186 - accuracy: 0.4184 - val_loss: 2.3681 - val_accuracy: 0.2870
Epoch 103/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9202 - accuracy: 0.4176 - val_loss: 2.3653 - val_accuracy: 0.2846
Epoch 104/110
237175/237175 [==============================] - 1s 6us/sample - loss:

237175/237175 [==============================] - 1s 6us/sample - loss: 2.0248 - accuracy: 0.3874 - val_loss: 2.3342 - val_accuracy: 0.2863
Epoch 42/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0203 - accuracy: 0.3887 - val_loss: 2.3230 - val_accuracy: 0.2887
Epoch 43/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0177 - accuracy: 0.3891 - val_loss: 2.3384 - val_accuracy: 0.2909
Epoch 44/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0153 - accuracy: 0.3908 - val_loss: 2.3532 - val_accuracy: 0.2853
Epoch 45/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0125 - accuracy: 0.3906 - val_loss: 2.3377 - val_accuracy: 0.2884
Epoch 46/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0091 - accuracy: 0.3921 - val_loss: 2.3403 - val_accuracy: 0.2892
Epoch 47/110
237175/237175 [==============================] - 1s 6us/sample - loss: 2.0058 - accuracy

Epoch 95/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9351 - accuracy: 0.4150 - val_loss: 2.3489 - val_accuracy: 0.2917
Epoch 96/110
237175/237175 [==============================] - 1s 5us/sample - loss: 1.9332 - accuracy: 0.4150 - val_loss: 2.3368 - val_accuracy: 0.2924
Epoch 97/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9315 - accuracy: 0.4149 - val_loss: 2.3450 - val_accuracy: 0.2904
Epoch 98/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9317 - accuracy: 0.4159 - val_loss: 2.3521 - val_accuracy: 0.2893
Epoch 99/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9298 - accuracy: 0.4162 - val_loss: 2.3464 - val_accuracy: 0.2888
Epoch 100/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.9297 - accuracy: 0.4159 - val_loss: 2.3715 - val_accuracy: 0.2883
Epoch 101/110
237175/237175 [==============================] - 1s 6us/sample - loss: 1.

,0
avg_val_f1_micro,0.289065
avg_val_f1_macro,0.280182
avg_val_accuracy,0.289065
avg_batch_val_accuracy,0.289064
avg_accuracy,0.384283
avg_val_loss,2.331053
avg_loss,2.037084
train_time,1553.272688


In [ ]:
ftrs_resampl, lbls_resampl = SMOTE().fit_resample(ftrs, lbl_encoded_lbls)
lbls_resampl_oh = one_hot_labels(lbls_resampl)

In [16]:
def train_final_model(x, y, run_name):
    
    save_path = constants.MODELS_PATH + run_name + '.h5' 
    
    input_shape = x[0].shape
    model = create_model(input_shape)
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    
    model.fit(
        x=x,
        y=y,
        epochs=110,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board],
        verbose=1
    )
    
    model.save(save_path)


train_final_model(ftrs_resampl, lbls_resampl_oh, 'base_mlp_marsyas_final')

Epoch 1/110
179555/179555 [==============================] - 3s 14us/sample - loss: 2.9494 - accuracy: 0.1338
Epoch 2/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.6211 - accuracy: 0.2082
Epoch 3/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.5307 - accuracy: 0.2327
Epoch 4/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.4850 - accuracy: 0.2468
Epoch 5/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.4536 - accuracy: 0.2559
Epoch 6/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.4280 - accuracy: 0.2642
Epoch 7/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.4047 - accuracy: 0.2713
Epoch 8/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.3850 - accuracy: 0.2749
Epoch 9/110
179555/179555 [==============================] - 1s 7us/sample - loss: 2.3698 - accuracy: 0.2777
Epoch 10/110
17955